In [0]:
# ________Inizialisiere Tensorflow/Keras_____
from __future__ import absolute_import, division, print_function

import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

In [0]:
# __________Download Daten_________
imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [0]:
# ___wie sehen die Daten aus(Bsp 1)? Zahl entspricht Text__
print(train_data[1])

In [0]:
#_____länge der Kommentare-->unterschiedlich____
len(train_data[0]), len(train_data[1])

In [0]:
#___Wieviele Daten? 25000 Kommentare; Label: Kommentare positiv 1 oder negativ 0__
print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))

In [0]:
#_____Integer zurück zu Text________

# Bibliothek
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [0]:
decode_review(train_data[0])

In [0]:
# _______Daten Normieren (sodass alle gleiche Länge)_____
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

In [0]:
# ______Länge nun gleich__________
len(train_data[0]), len(train_data[1])

In [0]:
print(train_data[0])

In [0]:
# _______Netz aufbauen___________
# 2 verschiedene Layer Embedding und Dense
# Embedding: neue Dimension; Globalaverage: Durchschnitt bilden
# input shape is the vocabulary count used for the movie reviews (10,000 words)
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

In [0]:
# _________Netz Compile_________
# Optimierung, Fehlerfunktion

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
# _________Validierungs Set erstellen_____
# Trainingsset, Validierungsset, Prüfungsset
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

In [0]:
# ___________Netz trainieren_______
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

In [0]:
# ________Netz bewerten mit Testdaten____
results = model.evaluate(test_data, test_labels)

print(results)

In [0]:
# ______Fehlerfunktion plotten_______
history_dict = history.history
history_dict.keys()

import matplotlib.pyplot as plt

acc = history_dict['acc']
val_acc = history_dict['val_acc']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [0]:
# ___________Genauigkeit Funktion plotten________
plt.clf()   # clear figure

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()